<a href="https://colab.research.google.com/github/franckepeixoto/FipeVeiculos/blob/main/FipeVeiculos_FranckePeixoto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://www.fipe.org.br/Content/img/logo_fipe.png' />

veiculos.fipe.org.br



In [ ]:
#@title FipeVeiculos v1 
#@markdown Consulta de Carros e Utilitários Pequenos
import requests
import pandas as pd
class FipeVeiculos:
  #------------------
  #
  # FipeVeiculos v1 - Consulta de Carros e Utilitários Pequenos
  # Informe primeiro a marca do veículo e depois o modelo e o ano modelo na ordem que desejar.
  # Você também pode digitar a informação no campo "busca", dentro de cada opção do formulário da pesquisa.
  # https://veiculos.fipe.org.br/api/veiculos/
  #
  # By Francke Peixoto
  #-------------------------------

  def __init__(self,codigoTipoVeiculo = 1):

    self.url_base = "https://veiculos.fipe.org.br/api/veiculos/"
    self.headers  = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36","From": "youremail@domain.com"}
    self.codigoTipoVeiculo = codigoTipoVeiculo
    self.tabelaReferencia = None
    self.marcas = None
    self.modelos = None
    self.tabela = {};
    self.tabelaAnoModelo = {};

  def ObterTabelaDeReferencia(self):

    if self.tabelaReferencia is None:
      response  = requests.post(self.url_base+'ConsultarTabelaDeReferencia', headers = self.headers)
      self.headers = requests.utils.default_headers()
      self.tabelaReferencia = response.json()
      self.tabelaReferencia = pd.DataFrame(data=self.tabelaReferencia)
    return self.tabelaReferencia

  def ObterMarcas(self,codigoTabelaReferencia):

    if self.marcas is None:
      payload = {}
      payload['codigoTipoVeiculo'] = self.codigoTipoVeiculo
      payload['codigoTabelaReferencia'] = codigoTabelaReferencia
      response  = requests.post(self.url_base+'ConsultarMarcas',payload, headers = self.headers)
      headers = requests.utils.default_headers() 
      self.marcas = response.json()
      self.marcas = pd.DataFrame(data=self.marcas)
    return self.marcas 

  def ObterModelos(self,codigoTabelaReferencia,codigoMarca):

    if self.modelos is None:
      payload = {}
      payload['codigoTipoVeiculo'] = self.codigoTipoVeiculo
      payload['codigoTabelaReferencia'] = codigoTabelaReferencia
      payload['codigoMarca'] = codigoMarca
      response  = requests.post(self.url_base+'ConsultarModelos',payload, headers = self.headers)
      headers = requests.utils.default_headers()
      self.modelos = response.json()['Modelos'] 
    return pd.DataFrame(data=self.modelos)

  def ObterAnoModelo(self,codigoTabelaReferencia,codigoMarca,codigoModelo):

    chave = str(codigoTabelaReferencia)+'_'+str(codigoMarca)+'_'+str(codigoModelo)
    if self.tabelaAnoModelo.get(chave) is None:
      payload = {}
      payload['codigoTipoVeiculo'] = self.codigoTipoVeiculo
      payload['codigoTabelaReferencia'] = codigoTabelaReferencia
      payload['codigoMarca'] = codigoMarca
      payload['codigoModelo'] = codigoModelo
      response  = requests.post(self.url_base+'ConsultarAnoModelo',payload, headers = self.headers)
      headers = requests.utils.default_headers() 
      self.anoModelos = response.json()
      self.anoModelos = pd.DataFrame(data=self.anoModelos)
      self.anoModelos['Value'] = self.anoModelos['Value'].transform(lambda v: v.split('-')[0]) 
      self.tabelaAnoModelo[chave] = self.anoModelos 
    return self.tabelaAnoModelo[chave]

  def __ajustarValores(self, rows):

    carros = pd.DataFrame(data=rows)
    carros.Valor = carros.Valor.replace('[R$]', '', regex=True).replace('[\.]', '', regex=True).replace('[\,]', '.', regex=True).astype(float)
    carros.AnoModelo = carros.AnoModelo.astype(int)
    dropColumns = ['TipoVeiculo','DataConsulta','Autenticacao']
    carros.drop(columns=dropColumns,inplace=True)
    return carros
    
  def ObterValores(self, codigoMarca,codigoModelo,limite=24):

    chave = str(codigoMarca)+'_'+str(codigoModelo)
    if self.tabela.get(chave) is None:
      self.ObterTabelaDeReferencia()
      self.rows =[]
      for referencia in  self.tabelaReferencia.values[:limite]:
        codigoTabelaReferencia = referencia[0] 
        anoModelos = self.ObterAnoModelo(codigoTabelaReferencia,codigoMarca,codigoModelo)
         #print('codigoTabelaReferencia',codigoTabelaReferencia)
        for anoModelo in anoModelos.values:
          codigoAnoModelo = anoModelo[1]
          payload = {}
          payload['codigoTabelaReferencia'] = codigoTabelaReferencia
          payload['codigoMarca'] = codigoMarca
          payload['codigoModelo'] = codigoModelo
          payload['codigoTipoVeiculo'] = self.codigoTipoVeiculo
          payload['anoModelo'] = codigoAnoModelo
          payload['codigoTipoCombustivel'] =1
          payload['tipoVeiculo']= 'carro'
          payload['modeloCodigoExterno'] =''
          payload['tipoConsulta'] = 'tradicional'
          response  = requests.post(self.url_base+'ConsultarValorComTodosParametros',payload, headers = self.headers)
          headers = requests.utils.default_headers() 
          row = response.json()
          if len(row) > 9:
            self.rows.append(row)
      self.tabela[chave] = self.__ajustarValores(self.rows) 
    return self.tabela[chave]
 

In [ ]:
fipe = FipeVeiculos()
codigoTabelaReferencia  = fipe.ObterTabelaDeReferencia()[:1].Codigo.values[0]
print(fipe.ObterTabelaDeReferencia()[:1],' >>>> ',codigoTabelaReferencia)

   Codigo          Mes
0     287  julho/2022   >>>>  287


In [ ]:
marcas = fipe.ObterMarcas(codigoTabelaReferencia)
nomes =['Ford','GM - Chevrolet','Fiat']
marcas[marcas.Label.str.strip().isin(nomes)].head()

,Label,Value
28,Fiat,21
30,Ford,22
34,GM - Chevrolet,23


In [ ]:
fipe.ObterTabelaDeReferencia().head()

,Codigo,Mes
0,287,julho/2022
1,286,junho/2022
2,285,maio/2022
3,284,abril/2022
4,283,março/2022


In [ ]:
codigoMarca = 23
modelos = fipe.ObterModelos(codigoTabelaReferencia,codigoMarca)
modelos

,Label,Value
0,A-10 2.5/4.1,926
1,A-10 De Luxe 2.5/4.1,927
2,A-20 Custom Std. CD/ De Luxe CD,928
3,A-20 Custom/ C-20 Luxe 4.1,929
4,A-20 Custom/ C-20 S 4.1,930
...,...,...
519,Zafira Elegance 2.0 MPFI FlexPower 8V 5p,1234
520,Zafira Elite 2.0 MPFI 16v 136cv 5p,1235
521,Zafira Elite 2.0 MPFI FlexPower 8V Aut,1236
522,Zafira Elite 2.0 MPFI FlexPower 8V 5p,1237


In [ ]:
codigoModelo = 4259
carros = fipe.ObterValores(codigoMarca,codigoModelo, limite=6)
carros.sort_values(['AnoModelo', 'MesReferencia'], ascending = [True, True])
carros.to_csv('carros.csv')
carros

,Valor,Marca,Modelo,AnoModelo,Combustivel,CodigoFipe,MesReferencia,SiglaCombustivel
0,34695.0,GM - Chevrolet,Zafira Expres. 2.0 MPFI FlexPower 5p Aut,2012,Gasolina,004335-4,julho de 2022,G
1,33167.0,GM - Chevrolet,Zafira Expres. 2.0 MPFI FlexPower 5p Aut,2011,Gasolina,004335-4,julho de 2022,G
2,32257.0,GM - Chevrolet,Zafira Expres. 2.0 MPFI FlexPower 5p Aut,2010,Gasolina,004335-4,julho de 2022,G
3,28950.0,GM - Chevrolet,Zafira Expres. 2.0 MPFI FlexPower 5p Aut,2009,Gasolina,004335-4,julho de 2022,G
4,28161.0,GM - Chevrolet,Zafira Expres. 2.0 MPFI FlexPower 5p Aut,2008,Gasolina,004335-4,julho de 2022,G
5,27393.0,GM - Chevrolet,Zafira Expres. 2.0 MPFI FlexPower 5p Aut,2007,Gasolina,004335-4,julho de 2022,G
6,34857.0,GM - Chevrolet,Zafira Expres. 2.0 MPFI FlexPower 5p Aut,2012,Gasolina,004335-4,junho de 2022,G
7,33900.0,GM - Chevrolet,Zafira Expres. 2.0 MPFI FlexPower 5p Aut,2011,Gasolina,004335-4,junho de 2022,G
8,32996.0,GM - Chevrolet,Zafira Expres. 2.0 MPFI FlexPower 5p Aut,2010,Gasolina,004335-4,junho de 2022,G
9,29282.0,GM - Chevrolet,Zafira Expres. 2.0 MPFI FlexPower 5p Aut,2009,Gasolina,004335-4,junho de 2022,G
